# Issue [#99](https://github.com/bsorrentino/langgraph4j/issues/99) by [zu1k](https://github.com/zu1k)

Verify "**Edge Update Logic Error During Subgraph Processing**" 

In [1]:
var userHomeDir = System.getProperty("user.home");
var localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
var langchain4jVersion = "1.0.1";
var langchain4jbeta = "1.0.1-beta6";
var langgraph4jVersion = "1.6-SNAPSHOT";

In [2]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

In [3]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
// %dependency /list-repos
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-langchain4j:\{langgraph4jVersion}
%dependency /add org.bsc.langgraph4j:langgraph4j-agent-executor:\{langgraph4jVersion}
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add dev.langchain4j:langchain4j-open-ai:\{langchain4jVersion}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.6-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-langchain4j:1.6-SNAPSHOT
Adding dependency org.bsc.langgraph4j:langgraph4j-agent-executor:1.6-SNAPSHOT
Adding dependency dev.langchain4j:langchain4j:1.0.1
Adding dependency dev.langchain4j:langchain4j-open-ai:1.0.1
Solving dependencies
Resolved artifacts count: 55
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.6-SNAPSHOT/langgraph4j-core-1.6-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/L

**Initialize Logger**

In [4]:
try( var file = new java.io.FileInputStream("./logging.properties")) {
    java.util.logging.LogManager.getLogManager().readConfiguration( file );
}

var log = org.slf4j.LoggerFactory.getLogger("AdaptiveRag");


## State declaration

In [5]:
import org.bsc.langgraph4j.action.NodeAction;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.StateGraph;

import static org.bsc.langgraph4j.StateGraph.END;
import static org.bsc.langgraph4j.StateGraph.START;
import static org.bsc.langgraph4j.action.AsyncEdgeAction.edge_async;
import static org.bsc.langgraph4j.action.AsyncNodeAction.node_async;

import java.util.Map;
import java.util.Optional;

class State extends AgentState {

    public State(Map<String, Object> initData) {
        super(initData);
    }

    public Optional<String> intent() {
        return  value("intent");
    }
}


## Subgraph definitions

In [6]:
// SubGraph1 Definition
StateGraph<State> subGraph1 = new StateGraph<>( State::new )
            .addNode("work", node_async(state -> Map.of("step", "work1")))
            .addEdge(START, "work")
            .addEdge("work", END)
            ;

// SubGraph2 Definition
StateGraph<State> subGraph2 = new StateGraph<>( State::new )
            .addNode("work", node_async(state -> Map.of("step", "work2")))
            .addEdge(START, "work")
            .addEdge("work", END)
            ;


## IntentRecognize node declaration

In [7]:
class IntentRecognizeNode implements NodeAction<State> {

    String intent;

    public void setIntent( String intent ) {
        this.intent = intent;
    }

    public String getIntent() {
        return intent;
    }

    @Override
    public Map<String, Object> apply(State state) {
        return Map.of( "intent", intent );
    }

}


## Using Subgraph as StateGraph

In [8]:
var intentRecognizeNode = new IntentRecognizeNode();

// MainGraph Definition
var graph = new StateGraph<>( State::new )
        .addNode("intent_recognize", node_async(intentRecognizeNode))
        .addNode("subAgent1", subGraph1 )
        .addNode("subAgent2", subGraph2 )
        .addEdge(START, "intent_recognize")
        .addConditionalEdges("intent_recognize",
                edge_async( state ->
                        state.intent().orElseThrow() ),
                Map.of("explain", "subAgent1",
                        "query", "subAgent2"
                )
        )
        .addEdge("subAgent1", END)
        .addEdge("subAgent2", END)
        ;

var workflow = graph.compile();

// System.out.println( workflow.getGraph( GraphRepresentation.Type.PLANTUML, "", false ));

// EXPLAIN
log.info( "EXPLAIN");
intentRecognizeNode.setIntent("explain");
for( var output : workflow.stream( Map.of("input", "explain me") ) ) {
       log.info( "{}", output );
}

// QUERY
log.info( "QUERY");
intentRecognizeNode.setIntent("query");
for( var output : workflow.stream( Map.of("input", "search for") ) ) {
        log.info( "{}", output );
}
         


EXPLAIN 
START 
NodeOutput{node=__START__, state={input=explain me}} 
NodeOutput{node=intent_recognize, state={input=explain me, intent=explain}} 
NodeOutput{node=subAgent1-work, state={input=explain me, intent=explain, step=work1}} 
NodeOutput{node=__END__, state={input=explain me, intent=explain, step=work1}} 
QUERY 
START 
NodeOutput{node=__START__, state={input=search for}} 
NodeOutput{node=intent_recognize, state={input=search for, intent=query}} 
NodeOutput{node=subAgent2-work, state={input=search for, intent=query, step=work2}} 
NodeOutput{node=__END__, state={input=search for, intent=query, step=work2}} 


## Using Subgraph as CompiledGraph

In [9]:
var intentRecognizeNode = new IntentRecognizeNode();

// MainGraph Definition
var graph = new StateGraph<>( State::new )
        .addNode("intent_recognize", node_async(intentRecognizeNode))
        .addNode("subAgent1", subGraph1.compile() )
        .addNode("subAgent2", subGraph2.compile() )
        .addEdge(START, "intent_recognize")
        .addConditionalEdges("intent_recognize",
                edge_async( state ->
                        state.intent().orElseThrow() ),
                Map.of("explain", "subAgent1",
                        "query", "subAgent2"
                )
        )
        .addEdge("subAgent1", END)
        .addEdge("subAgent2", END)
        ;

var workflow = graph.compile();

// System.out.println( workflow.getGraph( GraphRepresentation.Type.PLANTUML, "", false ));

// EXPLAIN
log.info( "EXPLAIN");
intentRecognizeNode.setIntent("explain");
for( var output : workflow.stream( Map.of("input", "explain me") ) ) {
       log.info( "{}", output );
}

// QUERY
log.info( "QUERY");
intentRecognizeNode.setIntent("query");
for( var output : workflow.stream( Map.of("input", "search for") ) ) {
        log.info( "{}", output );
}
         


EXPLAIN 
START 
NodeOutput{node=__START__, state={input=explain me}} 
START 
NodeOutput{node=intent_recognize, state={input=explain me, intent=explain}} 
NodeOutput{node=__START__, state={input=explain me, intent=explain}} 
NodeOutput{node=work, state={input=explain me, intent=explain, step=work1}} 
NodeOutput{node=__END__, state={input=explain me, intent=explain, step=work1}} 
NodeOutput{node=subAgent1, state={input=explain me, intent=explain, step=work1}} 
NodeOutput{node=__END__, state={input=explain me, intent=explain, step=work1}} 
QUERY 
START 
NodeOutput{node=__START__, state={input=search for}} 
START 
NodeOutput{node=intent_recognize, state={input=search for, intent=query}} 
NodeOutput{node=__START__, state={input=search for, intent=query}} 
NodeOutput{node=work, state={input=search for, intent=query, step=work2}} 
NodeOutput{node=__END__, state={input=search for, intent=query, step=work2}} 
NodeOutput{node=subAgent2, state={input=search for, intent=query, step=work2}} 
NodeOu